In [31]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas import Series, DataFrame
import pytz
from pytz import common_timezones, all_timezones
import matplotlib
matplotlib.style.use('ggplot')
%matplotlib inline
from datetime import datetime
import scipy as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf
matplotlib.style.use('fivethirtyeight')
matplotlib.style.use('seaborn-talk')
import os
import collections

In [32]:
preDouble = "\\\\iobsdc01\\SharedDocs\\SEER_MEDICARE\\SEER_MEDICARE_STAGING"
new = os.chdir(preDouble)
print(preDouble)

\\iobsdc01\SharedDocs\SEER_MEDICARE\SEER_MEDICARE_STAGING


In [33]:
import pickle
fileObject = open('control_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
controlindices = list(b)

In [34]:
import pickle
fileObject = open('treatment_indicies.pickle', 'rb')
b = pickle.load(fileObject)
fileObject.close()
treatmentindices = list(b)

In [35]:
print(len(controlindices), len(treatmentindices))

1031 1031


Want to get the concatenated outsaf files for only these patients.

In [36]:
import glob
#filelist = glob.glob('./outsaf*')
filelist = glob.glob('outsaf*')
goodfilelist = [f for f in filelist if f.endswith('.txt')]
goodfilelist[:15]

['outsaf06.file001.txt',
 'outsaf06.file002.txt',
 'outsaf06.file003.txt',
 'outsaf06.file004.txt',
 'outsaf06.file005.txt',
 'outsaf06.file006.txt',
 'outsaf06.file007.txt',
 'outsaf06.file008.txt',
 'outsaf06.file009.txt',
 'outsaf06.file010.txt',
 'outsaf06.file011.txt',
 'outsaf06.file012.txt',
 'outsaf06.file013.txt',
 'outsaf06.file014.txt',
 'outsaf06.file015.txt']

In [37]:
specsdict = {'patient_id': (0,0+11),
            'casereg': (0,0+2),
             'claim_from_date': (31,31+8),
             'claim_through_date': (39,39+8),
             'fac_type': (59,59+1),
             'freq_cd': (61,61+1),
             'pmt_amt': (64,64+15),              
             'tot_chrg': (79,79+15),
             'tot_line': (174,174+3),
             'seg_line': (177,177+2),
             'e1dgnscd': (198,198+7),
             'opsrvtyp': (216,216+1),
             'center': (218,218+4),
             'hcpcs': (240,240+5),
             'mf1': (245,245+2),
             'mf2': (247,247+2),
             'mf3': (249,249+2),
             'charge': (372,372+15),
             'revncvr': (402,402+15),
            'year': (421,421+4),
            'link_num': (425,425+10),
             'rec_count': (447,447+3),
             'tot_seg': (443,443+2),
             'seg_num': (445,445+2),
              'prvdrpmt': (495,495+15),
            'edgnsd1': (743,743+7),
            'edgnsd2': (750,750+7),
            'edgnsd3': (757,757+7),
            'prcdr_cd1': (785,785+7),
            'prcdr_cd2': (785+7, 785 + 2*7),
            'prcdr_cd3': (785+2*7, 785+3*7)}

names = sorted(specsdict,key=specsdict.__getitem__)

colspecs = [specsdict[n] for n in names]

patient_id, claim from date, linknum, and rec_count
is the key

In [38]:
df = pd.read_fwf(goodfilelist[1], colspecs = colspecs,
                names=names)

df.patient_id = df.patient_id.astype(str)
df.patient_id = df.patient_id.apply(lambda x: x.rjust(10, '0'))

df.set_index('patient_id', inplace=True,drop=False)

Need to just keep the subset of df that corresonds to the controlindices and treatmentindices.

In [39]:
df.shape

(520000, 31)

In [40]:
df.head(10)

,casereg,patient_id,claim_from_date,claim_through_date,fac_type,freq_cd,pmt_amt,tot_chrg,tot_line,seg_line,...,tot_seg,seg_num,rec_count,prvdrpmt,edgnsd1,edgnsd2,edgnsd3,prcdr_cd1,prcdr_cd2,prcdr_cd3
patient_id,,,,,,,,,,,,,,,,,,,,,
0263468060,2,0263468060,8222006,8222006,1,1,798.05,4337.20,7,7,...,1,1,7,798.05,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,1,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,2,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,3,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,10302006,10302006,7,2,90.03,219.00,4,4,...,1,1,4,90.03,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,1,88.07,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,2,88.07,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,3,88.07,NaN,NaN,NaN,NaN,NaN,NaN
0263468060,2,0263468060,4102006,4102006,1,3,88.07,423.25,6,6,...,1,1,4,88.07,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
df.hcpcs.value_counts()

36415    22884
85025    13831
97110    13436
85610    10116
80053     7887
77418     7605
80061     6668
82565     6632
84520     6186
97140     5448
77413     5123
77417     4775
80048     4593
80051     4530
77414     4526
88305     4396
84443     4304
71020     3949
85027     3747
93005     3701
82947     3643
84460     3626
77336     3581
84450     3531
99213     3398
83036     3393
97530     3095
85730     2695
99214     2665
97035     2658
         ...  
A4321        1
36100        1
83909        1
42826        1
84202        1
69320        1
99374        1
76100        1
75984        1
24505        1
31646        1
84588        1
86886        1
27340        1
74425        1
90376        1
73201        1
31717        1
87281        1
95925        1
42440        1
75860        1
90691        1
L3800        1
L3763        1
67255        1
97761        1
L3650        1
58353        1
38520        1
Name: hcpcs, dtype: int64

In [42]:
goodindices = controlindices + treatmentindices
print(len(goodindices),len(controlindices),len(treatmentindices))

2062 1031 1031


In [43]:
list(set(df.index).intersection(set(goodindices)))

['0292469070',
 '0268487040',
 '0280413070',
 '0299434050',
 '0272492080',
 '0274484060',
 '0267404090',
 '0273472060',
 '0277407050']

In [44]:
df.loc[list(set(df.index).intersection(set(goodindices)))].shape

(204, 31)

In [45]:
def get_good_outclaims(fin,goodindices):
    """Takes a file name, loads it and returns only
    the parts corresponding to goodindices."""
    df = pd.read_fwf(fin, colspecs = colspecs,
                names=names)
    df.patient_id = df.patient_id.astype(str)
    df.patient_id = df.patient_id.apply(lambda x: x.rjust(10, '0'))

    df.set_index('patient_id', inplace=True,drop=False)
    
    
    
    df = df.loc[list(set(df.index).intersection(set(goodindices)))]
    
    print(df.shape)
    print(list(set(df.index).intersection(set(goodindices))))
    
    return df

In [46]:
allframeslist = []

for i in range(len(goodfilelist)):
    di = get_good_outclaims(goodfilelist[i],goodindices)
    allframeslist.append(di)
    print(i, di.shape, len(allframeslist))
    del di

(239, 31)
['0244410011', '0247485010', '0261444090', '0259425030', '0231464080', '0242402011', '0260460050', '0247481030']
0 (239, 31) 1
(204, 31)
['0292469070', '0268487040', '0280413070', '0299434050', '0272492080', '0274484060', '0267404090', '0273472060', '0277407050']
1 (204, 31) 2
(0, 31)
[]
2 (0, 31) 3
(334, 31)
['2094125058', '2078111058', '2102136080', '2116168000', '2112160000', '2200086525', '2103177050']
3 (334, 31) 4
(167, 31)
['2261174000', '2269141070', '2257126090', '2238184000', '2264064545', '2281022515']
4 (167, 31) 5
(430, 31)
['2312190040', '2322078263', '2519009006', '2338115040', '2514038016', '2317138050', '2327193080', '2310153070', '2310165000', '2327074262', '2358169000']
5 (430, 31) 6
(673, 31)
['2584076045', '2588046095', '2557043065', '2578078015', '2557085075', '2562007085', '2606112030', '2583016095', '2595053065', '2582072045', '2576081025', '2539035016', '2556008005', '2606048013', '2535099046', '2581072085']
6 (673, 31) 7
(896, 31)
['4216095044', '424

Concatenate the above dataframe.

In [47]:
len(allframeslist)

218

In [48]:
a = allframeslist

In [49]:
dg = pd.concat(a, ignore_index=True)

In [50]:
dg.shape

(347817, 31)

In [51]:
dg.to_pickle('goodindices_costofcare_outsaf.pickle')

In [60]:
dg = pd.read_pickle('goodindices_costofcare_outsaf.pickle')

In [61]:
dg.head(15)

,casereg,patient_id,claim_from_date,claim_through_date,fac_type,freq_cd,pmt_amt,tot_chrg,tot_line,seg_line,...,tot_seg,seg_num,rec_count,prvdrpmt,edgnsd1,edgnsd2,edgnsd3,prcdr_cd1,prcdr_cd2,prcdr_cd3
0,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,1,984.10,NaN,NaN,NaN,NaN,NaN,NaN
1,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,2,984.10,NaN,NaN,NaN,NaN,NaN,NaN
2,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,3,984.10,NaN,NaN,NaN,NaN,NaN,NaN
3,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,4,984.10,NaN,NaN,NaN,NaN,NaN,NaN
4,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,5,984.10,NaN,NaN,NaN,NaN,NaN,NaN
5,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,6,984.10,NaN,NaN,NaN,NaN,NaN,NaN
6,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,7,984.10,NaN,NaN,NaN,NaN,NaN,NaN
7,2,0244410011,12192006,12192006,1,1,984.10,5242.49,8,8,...,1,1,8,984.10,NaN,NaN,NaN,NaN,NaN,NaN
8,2,0244410011,6242006,6242006,1,1,275.81,1142.80,8,8,...,1,1,1,275.81,NaN,NaN,NaN,NaN,NaN,NaN
9,2,0244410011,6242006,6242006,1,1,275.81,1142.80,8,8,...,1,1,2,275.81,NaN,NaN,NaN,NaN,NaN,NaN


In [62]:
dg.shape

(347817, 31)

In [63]:
krazykols = ['patient_id','claim_from_date',
            'link_num','rec_count','hcpcs','tot_chrg','tot_line',
            'seg_line']

dg[krazykols].head(30)

,patient_id,claim_from_date,link_num,rec_count,hcpcs,tot_chrg,tot_line,seg_line
0,0244410011,12192006,1042840,1,NaN,5242.49,8,8
1,0244410011,12192006,1042840,2,NaN,5242.49,8,8
2,0244410011,12192006,1042840,3,NaN,5242.49,8,8
3,0244410011,12192006,1042840,4,NaN,5242.49,8,8
4,0244410011,12192006,1042840,5,NaN,5242.49,8,8
5,0244410011,12192006,1042840,6,66984,5242.49,8,8
6,0244410011,12192006,1042840,7,NaN,5242.49,8,8
7,0244410011,12192006,1042840,8,NaN,5242.49,8,8
8,0244410011,6242006,1285544,1,NaN,1142.80,8,8
9,0244410011,6242006,1285544,2,80048,1142.80,8,8


In [64]:
dg.claim_from_date.head()

0    12192006
1    12192006
2    12192006
3    12192006
4    12192006
Name: claim_from_date, dtype: int64

In [65]:
dg.claim_from_date.dtype

dtype('int64')

In [66]:
dg['claim_from_date'] = dg['claim_from_date'].apply(str)
dg['claim_from_date'] = dg['claim_from_date'].apply(lambda x: x.rjust(8, '0'))

In [67]:
dg.claim_from_date.head(30)

0     12192006
1     12192006
2     12192006
3     12192006
4     12192006
5     12192006
6     12192006
7     12192006
8     06242006
9     06242006
10    06242006
11    06242006
12    06242006
13    06242006
14    06242006
15    06242006
16    09032006
17    09032006
18    09032006
19    10032006
20    10032006
21    10262006
22    10262006
23    01112006
24    01112006
25    01112006
26    01112006
27    01112006
28    01112006
29    01112006
Name: claim_from_date, dtype: object

In [68]:
dg.claim_from_date[10]

'06242006'

In [73]:

dg['claim_from_date'] = pd.to_datetime(dg['claim_from_date'], format='%m%d%Y')


In [74]:
dg['claim_from_date'].head(30)

0    2006-12-19
1    2006-12-19
2    2006-12-19
3    2006-12-19
4    2006-12-19
5    2006-12-19
6    2006-12-19
7    2006-12-19
8    2006-06-24
9    2006-06-24
10   2006-06-24
11   2006-06-24
12   2006-06-24
13   2006-06-24
14   2006-06-24
15   2006-06-24
16   2006-09-03
17   2006-09-03
18   2006-09-03
19   2006-10-03
20   2006-10-03
21   2006-10-26
22   2006-10-26
23   2006-01-11
24   2006-01-11
25   2006-01-11
26   2006-01-11
27   2006-01-11
28   2006-01-11
29   2006-01-11
Name: claim_from_date, dtype: datetime64[ns]

In [75]:
dg['claim_from_date_day'] = dg['claim_from_date'].dt.day
dg['claim_from_date_month'] = dg['claim_from_date'].dt.month
dg['claim_from_date_year'] = dg['claim_from_date'].dt.year

## Now need to load in the stuff from the pedsf big file.

We need to only compute total cost of care for each patient id, where the 'claim_from_date' is later in time than
the 'yrdx1' <= 'claim_from_date_year'

In [76]:
df = pd.read_pickle('dfpedsfFirsts.pickle')

In [78]:
df = df.set_index('patient_id',drop=False)

In [79]:
df_all_good_indices = df.loc[goodindices].copy()

In [80]:
df_all_good_indices.shape

(2062, 496)

In [81]:
df_all_good_indices.yrdx1.head(30)

patient_id
8835051335    2009
8880063314    2008
8865006343    2007
8833080450    2010
8887079350    2006
8855021414    2011
8834065023    2008
8876047420    2010
8840048462    2011
8874062373    2008
8880078384    2009
8808024403    2010
8886094334    2009
8839026363    2007
8887015421    2011
8827047330    2006
2269141070    2010
8897009480    2010
8878081394    2009
8819053490    2009
8861050420    2009
8884004490    2010
8850077480    2010
8827035335    2009
8871006401    2011
8865048335    2009
8882074493    2011
8888042432    2011
8813017314    2007
8822039322    2007
Name: yrdx1, dtype: int64

In [82]:
del df

In [85]:
monthdict = {'Jan': '01',
            'Feb': '02',
            'Mar': '03',
            'Apr': '04',
            'May': '05',
            'Jun': '06',
            'Jul': '07',
            'Aug': '08',
            'Sep': '09',
            'Oct': '10',
            'Nov': '11',
            'Dec': '12'}

In [86]:
df_all_good_indices['month1'] = df_all_good_indices['modx1'].replace(monthdict)
df_all_good_indices['month1'].head()

patient_id
8835051335    04
8880063314    12
8865006343    07
8833080450    07
8887079350    06
Name: month1, dtype: object

In [90]:
df_all_good_indices['month1'].isnull().sum()

8

In [91]:
df_all_good_indices = df_all_good_indices[~df_all_good_indices['month1'].isnull()]

In [92]:
df_all_good_indices['yrdx_plus_modx'] = df_all_good_indices.yrdx1.astype('str') + df_all_good_indices.month1.astype('str')

In [93]:
df_all_good_indices['yrdx_plus_modx'].head(30)

patient_id
8835051335    200904
8880063314    200812
8865006343    200707
8833080450    201007
8887079350    200606
8855021414    201104
8834065023    200810
8876047420    201001
8840048462    201107
8874062373    200804
8880078384    200903
8808024403    201011
8886094334    200902
8839026363    200704
8887015421    201103
8827047330    200604
2269141070    201005
8897009480    201005
8878081394    200903
8819053490    200910
8861050420    200907
8884004490    201004
8850077480    201001
8827035335    200903
8871006401    201101
8865048335    200903
8882074493    201110
8888042432    201104
8813017314    200712
8822039322    200708
Name: yrdx_plus_modx, dtype: object

In [94]:
# g['claim_from_date'] = pd.to_datetime(dg['claim_from_date'], format='%m%d%Y')

df_all_good_indices['yrdx_plus_modx'] = pd.to_datetime(df_all_good_indices['yrdx_plus_modx'], 
                                                      format='%Y%m')

In [95]:
df_all_good_indices['yrdx_plus_modx'].head(15)

patient_id
8835051335   2009-04-01
8880063314   2008-12-01
8865006343   2007-07-01
8833080450   2010-07-01
8887079350   2006-06-01
8855021414   2011-04-01
8834065023   2008-10-01
8876047420   2010-01-01
8840048462   2011-07-01
8874062373   2008-04-01
8880078384   2009-03-01
8808024403   2010-11-01
8886094334   2009-02-01
8839026363   2007-04-01
8887015421   2011-03-01
Name: yrdx_plus_modx, dtype: datetime64[ns]

Now we need to add this column to dg

In [97]:
dg = dg.set_index('patient_id',drop=False)

In [98]:
dh = dg.loc[df_all_good_indices.index]

In [99]:
dh.shape

(346507, 34)

In [100]:
df_all_good_indices.shape

(2054, 498)

In [101]:
dh['yrdx_plus_modx'] = df_all_good_indices['yrdx_plus_modx']

In [102]:
dh.head()

,casereg,patient_id,claim_from_date,claim_through_date,fac_type,freq_cd,pmt_amt,tot_chrg,tot_line,seg_line,...,edgnsd1,edgnsd2,edgnsd3,prcdr_cd1,prcdr_cd2,prcdr_cd3,claim_from_date_day,claim_from_date_month,claim_from_date_year,yrdx_plus_modx
patient_id,,,,,,,,,,,,,,,,,,,,,
8835051335,88.0,8835051335,2006-05-26,5262006.0,1.0,0,0.0,63.2,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,5.0,2006.0,2009-04-01
8835051335,88.0,8835051335,2006-05-26,5262006.0,1.0,0,0.0,63.2,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,5.0,2006.0,2009-04-01
8835051335,88.0,8835051335,2006-05-26,5262006.0,1.0,0,0.0,63.2,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,5.0,2006.0,2009-04-01
8835051335,88.0,8835051335,2006-05-26,5262006.0,1.0,0,0.0,63.2,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,5.0,2006.0,2009-04-01
8835051335,88.0,8835051335,2006-05-26,5262006.0,1.0,0,0.0,63.2,5.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,26.0,5.0,2006.0,2009-04-01


We only care about the costs for claims where ```claim_from_date``` is greater than or equal to ```yrdx_plus_modx```

In [104]:
krazykols = ['patient_id','claim_from_date','yrdx_plus_modx',
            'link_num','rec_count','hcpcs','tot_chrg','tot_line',
            'seg_line']

dh[krazykols].head(30)

,patient_id,claim_from_date,yrdx_plus_modx,link_num,rec_count,hcpcs,tot_chrg,tot_line,seg_line
patient_id,,,,,,,,,
8835051335,8835051335,2006-05-26,2009-04-01,69414.0,1.0,80053,63.20,5.0,5.0
8835051335,8835051335,2006-05-26,2009-04-01,69414.0,2.0,82465,63.20,5.0,5.0
8835051335,8835051335,2006-05-26,2009-04-01,69414.0,3.0,84153,63.20,5.0,5.0
8835051335,8835051335,2006-05-26,2009-04-01,69414.0,4.0,85025,63.20,5.0,5.0
8835051335,8835051335,2006-05-26,2009-04-01,69414.0,5.0,NaN,63.20,5.0,5.0
8880063314,8880063314,2007-12-15,2008-12-01,4455334.0,1.0,72040,1822.00,4.0,4.0
8880063314,8880063314,2007-12-15,2008-12-01,4455334.0,2.0,70450,1822.00,4.0,4.0
8880063314,8880063314,2007-12-15,2008-12-01,4455334.0,3.0,99284,1822.00,4.0,4.0
8880063314,8880063314,2007-12-15,2008-12-01,4455334.0,4.0,NaN,1822.00,4.0,4.0


In [106]:
(dh.claim_from_date >= dh.yrdx_plus_modx).sum()

296588

In [107]:
dh.shape

(346507, 35)

In [108]:
dh = dh[(dh.claim_from_date >= dh.yrdx_plus_modx)]

In [109]:
dhcontrol = dh.loc[controlindices]

In [111]:
dhtreatment = dh.loc[treatmentindices]

In [114]:
dhcontrol[krazykols].head(10)

,patient_id,claim_from_date,yrdx_plus_modx,link_num,rec_count,hcpcs,tot_chrg,tot_line,seg_line
patient_id,,,,,,,,,
8835051335,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,1.0,88305,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,2.0,88305,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,3.0,88342,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,4.0,88342,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,5.0,88342,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,6.0,NaN,457.69,6.0,6.0
8880063314,8880063314,2009-06-01,2008-12-01,4622850.0,1.0,77336,16280.00,19.0,19.0
8880063314,8880063314,2009-06-01,2008-12-01,4622850.0,2.0,77336,16280.00,19.0,19.0


In [116]:
dhcontrol.hcpcs.value_counts(normalize=True)[:10]

77418    0.074381
36415    0.047124
77421    0.030825
85025    0.024699
80053    0.019686
77014    0.018970
84153    0.018140
77336    0.017515
97110    0.015026
85610    0.014083
Name: hcpcs, dtype: float64

In [139]:
def make_hcpcs_dataframe(filehcpcs='OPTUM_HCPCS_BASE_2016_01_update.TXT'):
    """Make the dataframe corresponding to the hcpcs codes."""
    dfhcpcs = pd.read_fwf(filehcpcs,skip_blank_lines=True,header=None,widths=[5,2084],
        names=['code','description'])
    return dfhcpcs

In [140]:
def make_cpc_dataframe(filecpc = 'OPTUM_CPC_BASE_2016_01.TXT'):
    """Make the dataframe corresponding to the cpc (cpt) codes."""
    dfcpc = pd.read_fwf(filecpc,header=None,skip_blank_lines=True,
                     widths=[5,100],names=['code','description'])
    return dfcpc

In [141]:
def make_code_dataframe():
    """Combine the level1 and level2 code information into a single dataframe."""
    df1 = make_hcpcs_dataframe()
    df2 = make_cpc_dataframe()
    dftotal = df1[['code','description']].append(df2[['code','description']])
    return dftotal

In [142]:
def hcpcs_description_trans(df,dfhcpcs12,columnname):
    """Takes the dataframe df and the columnname as a string and adds
    a column containing the long description to df using the description string
    in dfhcpcs12."""
    dfhuh = pd.merge(df[[columnname]], dfhcpcs12,
                    left_on=columnname, right_on='code',how='left')
    df[columnname + '_description'] = dfhuh['description']
    del dfhuh

In [143]:
dfcodes = make_code_dataframe()

In [144]:
dfcodes.head(10)

,code,description
0,A0021,AMB SRVC OTSD STATE-MILE TRANSPORT AMB SERVICE...
1,A0080,NONEMERG TRNSPRT VOLUN NOT VESTED NONEMERG TR...
2,A0090,NONEMERG TRNSPRT IND W/VESTED INT NONEMERG TR...
3,A0100,NONEMERGENCY TRANSPORTATION; TAXI NONEMERGENC...
4,A0110,NONEMERG TRNSPRT&BUS INTERSTATE NONEMERG TR...
5,A0120,NONEMERG TRNSPRT: MINI-BUS MTN/OTH NONEMERG TR...
6,A0130,NONEMERG TRNSPRT: WHEELCHAIR VAN NONEMERGENC...
7,A0140,NONEMERG TRNSPRT & AIR TRAVEL NONEMERG TR...
8,A0160,NONEMERG TRNSPRT:MILE-CASE/SOCL WRKNONEMERG TR...
9,A0170,TRNSPRT ANCILLRY: PARK FEE TOLL OTHTRANSPORTAT...


In [145]:
hcpcs_description_trans(dhcontrol,dfcodes,'hcpcs')

In [146]:
hcpcs_description_trans(dhtreatment,dfcodes,'hcpcs')

In [147]:
list(dhcontrol.columns)

['casereg',
 'patient_id',
 'claim_from_date',
 'claim_through_date',
 'fac_type',
 'freq_cd',
 'pmt_amt',
 'tot_chrg',
 'tot_line',
 'seg_line',
 'e1dgnscd',
 'opsrvtyp',
 'center',
 'hcpcs',
 'mf1',
 'mf2',
 'mf3',
 'charge',
 'revncvr',
 'year',
 'link_num',
 'tot_seg',
 'seg_num',
 'rec_count',
 'prvdrpmt',
 'edgnsd1',
 'edgnsd2',
 'edgnsd3',
 'prcdr_cd1',
 'prcdr_cd2',
 'prcdr_cd3',
 'claim_from_date_day',
 'claim_from_date_month',
 'claim_from_date_year',
 'yrdx_plus_modx',
 'hcpcs_description',
 'description']

In [148]:
dhcontrol.hcpcs_description.value_counts(normalize=True)[:10]

Series([], Name: hcpcs_description, dtype: float64)

In [149]:
dhcontrol[krazykols]

,patient_id,claim_from_date,yrdx_plus_modx,link_num,rec_count,hcpcs,tot_chrg,tot_line,seg_line
patient_id,,,,,,,,,
8835051335,NaN,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,1.0,88305,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,2.0,88305,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,3.0,88342,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,4.0,88342,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,5.0,88342,457.69,6.0,6.0
8880063314,8880063314,2008-12-05,2008-12-01,1302752.0,6.0,NaN,457.69,6.0,6.0
8880063314,8880063314,2009-06-01,2008-12-01,4622850.0,1.0,77336,16280.00,19.0,19.0
8880063314,8880063314,2009-06-01,2008-12-01,4622850.0,2.0,77336,16280.00,19.0,19.0


In [150]:
dfcodes = dfcodes.set_index('code',drop=False)
dfcodes

,code,description
code,,
A0021,A0021,AMB SRVC OTSD STATE-MILE TRANSPORT AMB SERVICE...
A0080,A0080,NONEMERG TRNSPRT VOLUN NOT VESTED NONEMERG TR...
A0090,A0090,NONEMERG TRNSPRT IND W/VESTED INT NONEMERG TR...
A0100,A0100,NONEMERGENCY TRANSPORTATION; TAXI NONEMERGENC...
A0110,A0110,NONEMERG TRNSPRT&BUS INTERSTATE NONEMERG TR...
A0120,A0120,NONEMERG TRNSPRT: MINI-BUS MTN/OTH NONEMERG TR...
A0130,A0130,NONEMERG TRNSPRT: WHEELCHAIR VAN NONEMERGENC...
A0140,A0140,NONEMERG TRNSPRT & AIR TRAVEL NONEMERG TR...
A0160,A0160,NONEMERG TRNSPRT:MILE-CASE/SOCL WRKNONEMERG TR...


In [151]:
code_dict = dfcodes.description.to_dict()
#code_dict

In [152]:
dhcontrol['description'] = dhcontrol.hcpcs.replace(code_dict)

In [153]:
dhcontrol['description'].value_counts(normalize=True)[:10]

77418                                                                                                  0.074381
ROUTINE VENIPUNCTURE               COLLECTION VENOUS BLOOD VENIPUNCTURE            Collection of ve    0.047124
77421                                                                                                  0.030825
COMPLETE CBC W/AUTO DIFF WBC       BLOOD COUNT COMPLETE AUTO&AUTO DIFRNTL WBC      Blood count; com    0.024699
COMPREHEN METABOLIC PANEL          COMPREHENSIVE METABOLIC PANEL                   Comprehensive me    0.019686
CT SCAN FOR THERAPY GUIDE          CT GUIDANCE RADIATION THERAPY FLDS PLACEMENT    Computed tomogra    0.018970
ASSAY OF PSA TOTAL                 ASSAY OF PROSTATE SPECIFIC ANTIGEN TOTAL        Prostate specifi    0.018140
RADIATION PHYSICS CONSULT          CONTINUING MEDICAL PHYSICS CONSLTJ PR WK        Continuing medic    0.017515
THERAPEUTIC EXERCISES              THERAPEUTIC PX 1/> AREAS EACH 15 MIN EXERCISES  Therapeutic proc    0

In [154]:
dhtreatment['description'] = dhtreatment.hcpcs.replace(code_dict)

In [155]:
dhtreatment['description'].value_counts(normalize=True)[:10]

PROTON TRMT SIMPLE W/COMP          PROTON TX DELIVERY SIMPLE W/COMPENSATION        Proton treatment    0.142885
ROUTINE VENIPUNCTURE               COLLECTION VENOUS BLOOD VENIPUNCTURE            Collection of ve    0.040248
RADIATION PHYSICS CONSULT          CONTINUING MEDICAL PHYSICS CONSLTJ PR WK        Continuing medic    0.036325
COMPLETE CBC W/AUTO DIFF WBC       BLOOD COUNT COMPLETE AUTO&AUTO DIFRNTL WBC      Blood count; com    0.020621
RADIATION TREATMENT AID(S)         TX DEVICES DESIGN & CONSTRUCTION COMPLEX        Treatment device    0.016297
ASSAY OF PSA TOTAL                 ASSAY OF PROSTATE SPECIFIC ANTIGEN TOTAL        Prostate specifi    0.016139
PROTON TRMT INTERMEDIATE           PROTON TX DELIVERY INTERMEDIATE                 Proton treatment    0.015183
COMPREHEN METABOLIC PANEL          COMPREHENSIVE METABOLIC PANEL                   Comprehensive me    0.014307
DIALYSIS PROCEDURE                 UNLISTED DIALYSIS PROCEDURE INPATIENT/OUTPATIENTUnlisted dialysi    0